# **Extracting Features and converting it into a CSV file**

In [1]:
!pip install spotipy #Installing Spotipy

In [2]:
!pip install pandas #Installing Pandas

## Importing Necessary Libraries

In [3]:
import csv
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from csv import DictWriter

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id="", client_secret="")
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)  #Using Credentials to Log In

## Get Spoti Data Function

In [5]:
from numpy import array, array_split

def get_spoti_data(uris, uris_info, uris_artist):

    artist_data = []

    # tenemos que saber cuantas partes tenemos que dividir la lista de artistas en chunks de 50
    len_artist = len(uris_artist)


    if len_artist > 50:

        #numero de partes en las que se va a cortar la lista
        num_slices = (len_artist // 50) 

        #numero de elementos restantes
        mod = (len_artist % 50) 
        #mod = len_artist - mod

        if mod != 0:

            mod -= 1
            num_slices += 1

            #lista completa
            c_l = uris_artist[:-mod]

            #lista restante
            c_m = uris_artist[-mod:]
        
            #doing the split part
            splits = array(c_l)
            splits = array_split(c_l, num_slices)
            splits.append(c_m)
        
        else:
            splits = array(uris_artist)
            splits = array_split(splits, num_slices)

            

        for _array in splits:

            if isinstance(_array, np.ndarray):
                artists = _array.tolist()

            data = sp.artists(artists)

            #print(data)

            artist_data += [
                    {
                        "genre": genre["genres"][0]
                    }
                    for genre in data["artists"] if genre["genres"]]
    else:

        data = sp.artists(uris_artist)

        #print(data)


        artist_data += [
        {
            "genre": genre["genres"][0]
        }
        for genre in data["artists"] if genre["genres"] ]




    audio_feats = sp.audio_features(uris)


    #audio_analysis
    list_au = []
    for uri in uris:
        au_an = sp.audio_analysis(uri)
        au_an = {
            "bars": len(au_an["bars"]),
            "beats": len(au_an["beats"]),
            "sections": len(au_an["beats"]),
            "segments": len(au_an["segments"]),
            "tatums": len(au_an["tatums"]),
            **au_an["track"]
        }
        list_au.append(au_an)



    merge = [*zip(uris_info, audio_feats, artist_data, list_au)]



    f_process = [
        {
            **data_song, 
            **af,
            **genre,
            **aa
        }

        for data_song, af, genre, aa  in merge
    ]
    
    return f_process

    
    

## Creating File Function

In [6]:
def write(filename, data, writeheader):
    
        
    # Open CSV file in append mode
    # Create a file object for this file
    with open(filename, 'a', encoding="utf-8", newline='') as f_object:
    
        # Pass the file object and a list
        # of column names to DictWriter()
        # You will get a object of DictWriter
        dictwriter_object = DictWriter(f_object, fieldnames=[*data.keys()])

        if writeheader: dictwriter_object.writeheader()

    
        # Pass the dictionary as an argument to the Writerow()
        dictwriter_object.writerow(data)

    
        # Close the file object
        f_object.close()
    

## Recollect Data Function

In [7]:
def recollect(list_playlists, filename):

    # los grupos de playlist a mergear
    writed_headeder = False

    for playlists, playlist_name in list_playlists:
        print(playlist_name)

        for playlist in playlists:


            try:


                playlist_data = sp.playlist(playlist)
                
                tracks = playlist_data["tracks"]["items"]

                tracks_id = [
                    track["track"]["uri"]
                    for track in tracks
                ]

                artist_id = [
                    track["track"]["artists"][0]["uri"]
                    for track in tracks
                ]


                info_song = [
                    {
                        "song_name": track["track"]["name"],
                        "artist": track["track"]["artists"][0]["name"],
                        "playlist_name": playlist_name,
                        #"genre":  track["track"]["album"]["genres"][0]
                    }

                    for track in tracks
                ]


                songs = get_spoti_data(tracks_id, info_song, artist_id)

                for song in songs:
                    
                    write(filename, song, writeheader= True if writed_headeder is False else False)
                    writed_headeder=True
                    
            except Exception as e:
                print(e)


                



## Using Recollect Function 

### Using Recollect For Perreo Rock Corridos Cumbia

In [11]:
recollect(
    [
       (
            [
            "37i9dQZF1DWZjqjZMudx9T",
            "37i9dQZF1DWY7IeIP1cdjF",
            "37i9dQZF1DWX5ZOsG2Ogi1",
            "37i9dQZF1DWUlCmB8llCTB",
            "37i9dQZF1DX8SfyqmSFDwe",
            "37i9dQZF1DWVOLXBOylBUZ",
            "37i9dQZF1DWZdsS73T1ogG",
            "37i9dQZF1DX2EiUAL7Wdfc",
            ],
            "Regeaton"
       ),
       (
            [
            "37i9dQZF1DXcF6B6QPhFDv",
            "37i9dQZF1DWVqJMsgEN0F4",
            "37i9dQZF1DWZryfp6NSvtz",
            "37i9dQZF1DX82GYcclJ3Ug",
            "37i9dQZF1DWT2jS7NwYPVI",
            "37i9dQZF1DWYmDNATMglFU",
            "37i9dQZF1DWWJOmJ7nRx0C",
            "37i9dQZF1DX5a39zUmS7Hn",
            ],
            "Rock"
        ),
        (
            [
            "37i9dQZF1DX2shzuwwKw0y",
            "37i9dQZF1DWWhMyyrqZlaY",
            "37i9dQZF1DX8yZMfjCxaJx",
            "37i9dQZF1DWUuQG5KIiBcn",
            "37i9dQZF1DXc2p8xjyOmiI",
            "37i9dQZF1DXdDoYRQ4LfQJ",
            "37i9dQZF1DWU43UtgmzCjN",
            "37i9dQZF1DXaRkwbkfuyXo",
            ],
            "Corridos"
        ),
        (
            [
            "37i9dQZF1DX8yLfjPY8emY",
            "37i9dQZF1DX8yLfjPY8emY",
            "37i9dQZF1DWYpQJx6k6xat",
            "37i9dQZF1DWT1viuVscXm2",
            "37i9dQZF1DX02rJzEVvd4Q",
            "37i9dQZF1DWSLzWG0oani3",
            "37i9dQZF1DX8DBcbHcgPJw",
            "37i9dQZF1DX5463MYlHXHD",
            ],
            "Cumbia"
        ),
    ],
    "PerreoRockCorridosCumbia.csv"
)

Regeaton
'NoneType' object is not subscriptable
Rock
Corridos
Cumbia


### Using Recollect For Perreo Rock Corridos

In [9]:
recollect(
    [
       (
            [
            "37i9dQZF1DWZjqjZMudx9T",
            "37i9dQZF1DWY7IeIP1cdjF",
            "37i9dQZF1DWX5ZOsG2Ogi1",
            "37i9dQZF1DWUlCmB8llCTB",
            "37i9dQZF1DX8SfyqmSFDwe",
            "37i9dQZF1DWVOLXBOylBUZ",
            "37i9dQZF1DWZdsS73T1ogG",
            "37i9dQZF1DX2EiUAL7Wdfc",
            ],
            "Regeaton"
       ),
       (
            [
            "37i9dQZF1DXcF6B6QPhFDv",
            "37i9dQZF1DWVqJMsgEN0F4",
            "37i9dQZF1DWZryfp6NSvtz",
            "37i9dQZF1DX82GYcclJ3Ug",
            "37i9dQZF1DWT2jS7NwYPVI",
            "37i9dQZF1DWYmDNATMglFU",
            "37i9dQZF1DWWJOmJ7nRx0C",
            "37i9dQZF1DX5a39zUmS7Hn",
            ],
            "Rock"
        ),
        (
            [
            "37i9dQZF1DX2shzuwwKw0y",
            "37i9dQZF1DWWhMyyrqZlaY",
            "37i9dQZF1DX8yZMfjCxaJx",
            "37i9dQZF1DWUuQG5KIiBcn",
            "37i9dQZF1DXc2p8xjyOmiI",
            "37i9dQZF1DXdDoYRQ4LfQJ",
            "37i9dQZF1DWU43UtgmzCjN",
            "37i9dQZF1DXaRkwbkfuyXo",
            ],
            "Corridos"
        ),
    ],
    "PerreoRockCorridos.csv"
)

Regeaton
Rock
Corridos


### Using Recollect For Perreo Rock 

In [18]:
recollect(
    [
       (
            [
            "37i9dQZF1DWZjqjZMudx9T",
            "37i9dQZF1DWY7IeIP1cdjF",
            "37i9dQZF1DWX5ZOsG2Ogi1",
            "37i9dQZF1DWUlCmB8llCTB",
            "37i9dQZF1DX8SfyqmSFDwe",
            "37i9dQZF1DWVOLXBOylBUZ",
            "37i9dQZF1DWZdsS73T1ogG",
            "37i9dQZF1DX2EiUAL7Wdfc",
            ],
            "Regeaton"
       ),
       (
            [
            "37i9dQZF1DXcF6B6QPhFDv",
            "37i9dQZF1DWVqJMsgEN0F4",
            "37i9dQZF1DWZryfp6NSvtz",
            "37i9dQZF1DX82GYcclJ3Ug",
            "37i9dQZF1DWT2jS7NwYPVI",
            "37i9dQZF1DWYmDNATMglFU",
            "37i9dQZF1DWWJOmJ7nRx0C",
            "37i9dQZF1DX5a39zUmS7Hn",
            ],
            "Rock"
        ),
    ],
    "PerreoRock.csv"
)


Regeaton
HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)
Rock


### Using Recollect For ALL the Genres

In [11]:
recollect(
    [
             (
            [
            "37i9dQZF1DWX6Kn8siboHN",
            "37i9dQZF1DWSRuK6ajtwlC",
            "37i9dQZF1DX5b2iE2KAwwS",
            "37i9dQZF1DWU6iy0O1kuqg",
            "37i9dQZF1DX7V3ptrxki0a",
            "37i9dQZF1DXaJEFcu36oGH",
            "37i9dQZF1DX3LJg0ziYmU4",
            ],
            "Norteñas"
        ),
             (
            [
            "37i9dQZF1DX2shzuwwKw0y",
            "37i9dQZF1DWWhMyyrqZlaY",
            "37i9dQZF1DX8yZMfjCxaJx",
            "37i9dQZF1DWUuQG5KIiBcn",
            "37i9dQZF1DXc2p8xjyOmiI",
            "37i9dQZF1DXdDoYRQ4LfQJ",
            "37i9dQZF1DWU43UtgmzCjN",
            "37i9dQZF1DXaRkwbkfuyXo",
            ],
            "Corridos"
        ),
             (
            [
            "37i9dQZF1DX37tvjlT1GSK",
            "37i9dQZF1DX0P1EPhWUeqB",
            "37i9dQZF1DWZQGZ7yvpH00",
            "37i9dQZF1DWYNz9xcyse7Z",
            ],
            "Sierreño"
        ),
             (
            [
            "37i9dQZF1DXaaA14YcUIMA",
            "37i9dQZF1DWWTh1OeV2Zbn",
            "37i9dQZF1DWSKj6yrmnSvg",
            "37i9dQZF1DXaHka4CNUQvp",
            "37i9dQZF1DWYP6zmGbwfYH",
            "37i9dQZF1DX8VyhWnVrn9d",
            ],
            "Grupero"
        ),
             (
            [
            "37i9dQZF1DXaaA14YcUIMA",
            "37i9dQZF1DWZLL3REk8t1E",
            "37i9dQZF1DX6EM3TTcpK4b",
            "37i9dQZF1DX53wq0WYtg5L",
            "37i9dQZF1DXdC7eRcOJUCw",
            "37i9dQZF1DXcF3YtBdCala",
            "37i9dQZF1DXaqZ3USzRiOk",
            "37i9dQZF1DXa5GdOk4Dsb5",
            ],
            "Mariachi"
        ),
             (
            [
            "37i9dQZF1DWZjqjZMudx9T",
            "37i9dQZF1DWY7IeIP1cdjF",
            "37i9dQZF1DWX5ZOsG2Ogi1",
            "37i9dQZF1DWUlCmB8llCTB",
            "37i9dQZF1DX8SfyqmSFDwe",
            "37i9dQZF1DWVOLXBOylBUZ",
            "37i9dQZF1DWZdsS73T1ogG",
            "37i9dQZF1DX2EiUAL7Wdfc",
            ],
            "Regeaton"
        ),
             (
            [
            "37i9dQZF1DWSpF87bP6JSF",
            "37i9dQZF1DX1QnNyJOBQBv",
            "37i9dQZF1DX8womvTyUjrN",
            "37i9dQZF1DX6ThddIjWuGT",
            "37i9dQZF1DX3omIq8ziEt6",
            "37i9dQZF1DX7Qo2zphj7u3",
            "37i9dQZF1DWVpWZiQLVZtt",
            "37i9dQZF1DX05XOaKL5GVe",
            ],
            "Latin Pop"
        ),
             (
            [
            "37i9dQZF1DX7MTlMMRl0MD",
            "37i9dQZF1DX1h2mzk545UU",
            "37i9dQZF1DX65py6HnnlE1",
            "37i9dQZF1DWYNSDUeK85ev",
            ],
            "Tropical"
        ),

             (
            [
            "37i9dQZF1DX8yLfjPY8emY",
            "37i9dQZF1DX8yLfjPY8emY",
            "37i9dQZF1DWYpQJx6k6xat",
            "37i9dQZF1DWT1viuVscXm2",
            "37i9dQZF1DX02rJzEVvd4Q",
            "37i9dQZF1DWSLzWG0oani3",
            "37i9dQZF1DX8DBcbHcgPJw",
            "37i9dQZF1DX5463MYlHXHD",
            ],
            "Cumbia"
        ),
             (
            [
            "37i9dQZF1DWUFmUp1ez7YB",
            "37i9dQZF1DX2piJKuRdKIA",
            "37i9dQZF1DX1UHxedJfnRM",
            "37i9dQZF1DX0ERdy9Swdy5",
            "37i9dQZF1DX49mfoIGumhI",
            "37i9dQZF1DX2zJGRjWhK7Q",
            "37i9dQZF1DWVSSKiqqNKSC",
            "37i9dQZF1DX18u16ebmQgx",
            ],
            "Salsa"
        ),
             (
            [
            "37i9dQZF1DWVA1Gq4XHa6U",
            "37i9dQZF1DX186v583rmzp",
            "37i9dQZF1DWYGxBNe4qojI",
            "37i9dQZF1DX9sQDbOMReFI",
            "37i9dQZF1DX7Mq3mO5SSDc",
            "37i9dQZF1DWYok9l1JL7GM",
            "37i9dQZF1DWTJzNR1J5ygQ",
            "37i9dQZF1DWUFmyho2wkQU",
            ],
            "Classic Hip Hop"
        ),
             (
            [
            "37i9dQZF1DWVFZbCNivv6B",
            "37i9dQZF1DWTe5ECpTKzMo",
            "37i9dQZF1DX40O80BIPklU",
            "37i9dQZF1DX8IQoWYgeO8l",
            "37i9dQZF1DXbMnDeC4VENb",
            "37i9dQZF1DX4O0AdBo80OE",
            "37i9dQZF1DXc5PHJQUwAnG",
            "37i9dQZF1DXbTQNLzhgvdh",
            ],
            "Mexican Hip Hop"
        ),
             (
            [
            "37i9dQZF1DX2R0a3scWaq6",
            "37i9dQZF1DX1qNZsqIInBz",
            "37i9dQZF1DX6VdMW310YC7",
            "37i9dQZF1DX9pP7cC2liKw",
            "37i9dQZF1DX0r3x8OtiwEM",
            "37i9dQZF1DX55dNU0PWnO5",
            ],
            "Electronica & Chill"
        ),
             (
            [
            "37i9dQZF1DWWY64wDtewQt",
            "37i9dQZF1DXdDh4h59PJIQ",
            "37i9dQZF1DXcZDD7cfEKhW",
            "37i9dQZF1DWSf2RDTDayIx",
            "37i9dQZF1DXaXB8fQg7xif",
            ],
            "EDM & Dance Pop"
        ),
             (
            [
            "37i9dQZF1DX2TRYkJECvfC",
            "37i9dQZF1DX0AMssoUKCz7",
            "37i9dQZF1DX82pCGH5USnM",
            "37i9dQZF1DWZeXxACCcUmL",
            "37i9dQZF1DWSlskcBvOTt0",
            "37i9dQZF1DX8wtrGDH81Oa",
            ],
            "House"
        ),
             (
            [
            "37i9dQZF1DX6J5NfMJS675",
            "37i9dQZF1DX0r3x8OtiwEM",
            "37i9dQZF1DX8AliSIsGeKd",
            "37i9dQZF1DX36TRAnIL92N",
            ],
            "Techno"
        ),
             (
            [
            "37i9dQZF1DX1MUPbVKMgJE",
            "37i9dQZF1DX2GKumqRIZ7g",
            "37i9dQZF1DX6eVMzZ9dfKv",
            "37i9dQZF1DXbS8bPVXXR2B",
            "37i9dQZF1DXcAiMO5AYxE8",
            "37i9dQZF1DX9ZAByPehUEa",
            "37i9dQZF1DX1tW4VlEfDSS",
            ],
            "Disco"
        ),
             (
            [
            "37i9dQZF1DXcF6B6QPhFDv",
            "37i9dQZF1DWVqJMsgEN0F4",
            "37i9dQZF1DWZryfp6NSvtz",
            "37i9dQZF1DX82GYcclJ3Ug",
            "37i9dQZF1DWT2jS7NwYPVI",
            "37i9dQZF1DWYmDNATMglFU",
            "37i9dQZF1DWWJOmJ7nRx0C",
            "37i9dQZF1DX5a39zUmS7Hn",
            ],
            "Rock"
        ),
             (
            [
            "37i9dQZF1DXaaU1AaHpZeu",
            "37i9dQZF1DX68kWwWBDPYS",
            "37i9dQZF1DWZ0Zsena4JFh",
            "37i9dQZF1DXecqWg9vXl0U",
            "37i9dQZF1DX2xKqsL1SVWb",
            "37i9dQZF1DX2xhsrO0GW1M",
            "37i9dQZF1DXe4qIBASHIrI",
            "37i9dQZF1DX8HSZzYIDgBB",
            ],
            "Indie"
        ),
             (
            [
            "37i9dQZF1DWTcqUzwhNmKv",
            "37i9dQZF1DX5J7FIl4q56G",
            "37i9dQZF1DXdiUbJTV2anj",
            "37i9dQZF1DWXIcbzpLauPS",
            "37i9dQZF1DWY5ai7gxfuaS",
            "37i9dQZF1DWXDJDWnzE39E",
            "37i9dQZF1DX5wgKYQVRARv",
            "37i9dQZF1DX1cJWWyylDuw",
            ],
            "Metal"
        ),
             (
            [
            "37i9dQZF1DXasneILDRM7B",
            "37i9dQZF1DX3LDIBRoaCDQ",
            "37i9dQZF1DX9wa6XirBPv8",
            "37i9dQZF1DXaDPSVSkBhhM",
            "37i9dQZF1DX4mnAzYEyRVc",
            "37i9dQZF1DX3MU5XUozve7",
            "37i9dQZF1DXdRLa2gxBNsq",
            "37i9dQZF1DXd6tJtr4qeot",
            ],
            "Punk"

        ),
             (
            [
            "37i9dQZF1DX9uKNf5jGX6m",
            "37i9dQZF1DX57hHpm3vGiU",
            "37i9dQZF1DX8NxFcG0RivQ",
            "37i9dQZF1DX3AqNtukWhcT",
            "37i9dQZF1DWZvpVE2NxPV2",
            "37i9dQZF1DX0YDqnWWzvxD",
            "37i9dQZF1DX7R7Bjxm48PR",
            "37i9dQZF1DWSjkhfyRmxay",
             

            ],
            "Yoga"
        ),
             (
            [
            "37i9dQZF1DXadOVCgGhS7j",
            "37i9dQZF1DWWPcvnOpPG3x",
            "37i9dQZF1DWV3VLITCZusq",
            "37i9dQZF1DXbcCmWfoUaw2",
            "37i9dQZF1DWUxdwkOJZYCJ",
            "37i9dQZF1DX4Y1uAfxGdKJ",
            "37i9dQZF1DX8E1Op3UZWf0",
            "37i9dQZF1DX35oM5SPECmN",
             

            ],
            "Running"
        ),
             (
            [
            "37i9dQZF1DXbITWG1ZJKYt",
            "37i9dQZF1DWTR4ZOXTfd9K",
            "37i9dQZF1DX9GSZDbrndTa",
            "37i9dQZF1DWZodX1hXiq9y",
            "37i9dQZF1DWWp3UMHO0b0B",
            "37i9dQZF1DWXj0W55phbFm",
            "37i9dQZF1DX5S7hzwhDSyV",
            "37i9dQZF1DWSSSls9eK29h",
             

            ],
            "Jazz Classics"
        ),
             (
            [
            "37i9dQZF1DX4H3NUOwVLXj",
            "37i9dQZF1DZ06evO3XRXXy",
            "37i9dQZF1DX8OYzU0lx5hL",
            "37i9dQZF1DZ06evO0Co11u",
            "37i9dQZF1DZ06evO3h8eGI",
            "37i9dQZF1DX3BqHI5fuXWV",
            "37i9dQZF1DZ06evO200ybe",
            "37i9dQZF1DZ06evO17hgFa",
             

            ],
            "Jazz Icons"
        ),
             (
            [
            "37i9dQZF1DWSV3Tk4GO2fq",
            "37i9dQZF1DWXcg95telZlE",
            "37i9dQZF1DWSwFS0Z6E1ep",
            "37i9dQZF1DWUGhrXBsyMVJ",
            "37i9dQZF1DX3EgfgehUd1l",
            "37i9dQZF1DXe2fHCBYbwN7",
             

            ],
            "50's"
        ),
             (
            [
            "37i9dQZF1DXaKIA8E7WcJj",
            "37i9dQZF1DWWzBc3TOlaAV",
            "37i9dQZF1DWYzKmy0vGGcY",
            "37i9dQZF1DX3AdAEX3vkB1",
            "37i9dQZF1DX9TCQkJ8ktzo",
            "37i9dQZF1DWVAWtyHQImlQ",
             

            ],
            "60's"
        ),
             (
            [
            "37i9dQZF1DWTJ7xPn4vNaz",
            "37i9dQZF1DWWwzidNQX6jx",
            "37i9dQZF1DX1Hya1sRqqxI",
            "37i9dQZF1DWWiDhnQ2IIru",
            "37i9dQZF1DWTTn6daQVbOa",
            "37i9dQZF1DWSWNiyXQAvbl",
             

            ],
            "70's"
        ),
             (
            [
            "37i9dQZF1DX4UtSsGT1Sbe",
            "37i9dQZF1DX1spT6G94GFC",
            "37i9dQZF1DX6xnkAwJX7tn",
            "37i9dQZF1DX4WELsJtFZjZ",
            "37i9dQZF1DWYAcBZSAVhlf",
            "37i9dQZF1DX6l1fwN15uV5",
             

            ],
            "80's"
        ),
             (
            [
            "37i9dQZF1DXbTxeAdrVG2l",
            "37i9dQZF1DX802IXCAaWtY",
            "37i9dQZF1DXdo6A3mWpdWx",
            "37i9dQZF1DX1rVvRgjX59F",
            "37i9dQZF1DX2syo5w7a1cu",
            "37i9dQZF1DWV8xrpik0esU",
             

            ],
            "90's"
        ),
             (
            [
            "37i9dQZF1DX4o1oenSJRJd",
            "37i9dQZF1DX7e8TjkFNKWH",
            "37i9dQZF1DXdGHPXiRsW3u",
            "37i9dQZF1DWYmmr74INQlb",
            "37i9dQZF1DWSj4n2q6ZYVe",
            "37i9dQZF1DX8j4KHUVrE2f",
             

            ],
            "00's"
        ),
             (
            [
            "37i9dQZF1DX5Ejj0EkURtP",
            "37i9dQZF1DX99DRG9N39X3",
            "37i9dQZF1DWXbttAJcbphz",
            "37i9dQZF1DWWylYLMvjuRG",
            "37i9dQZF1DWWCk4v3ohhHQ",
            "37i9dQZF1DX7udthHrZCWz",
             

            ],
            "10's"
        ),
             (
            [
            "37i9dQZF1DX4s3V2rTswzO",
            "37i9dQZF1DX1lskIHz70Ye",
            "37i9dQZF1DXct2NyIphYfn",
            "37i9dQZF1DX6tTW0xDxScH",
            "37i9dQZF1DWT1R6bXL4dyW",
            "37i9dQZF1DWSUA8piie2LD",
             

            ],
            "Romance"
        ),
             (
            [
            "37i9dQZF1DXbEm2sKzgoJ8",
            "37i9dQZF1DWVGy1YP1ojM5",
            "37i9dQZF1DX1BGjF5N5O7F",
            "37i9dQZF1DXdl6IPOySdX4",
            "37i9dQZF1DWTDkmaK3cAib",
            "37i9dQZF1DX9kThPpg1lIF",
             

            ],
            "Intimacy"
        ),
             (
            [
            "37i9dQZF1DX4adj7PFEBwf",
            "37i9dQZF1DX3Z99viCDp7Q",
            "37i9dQZF1DXbTR7abB7Ji7",
            "37i9dQZF1DWZIjPTO9UrdW",
            "37i9dQZF1DX4YLk1hHv7S9",
            "37i9dQZF1DX3qS3zyciZpU",
             

            ],
            "Wedding"
        ),
             (
            [
            "37i9dQZF1DX46zHlTTyolY",
            "37i9dQZF1DWSqBruwoIXkA",
            "37i9dQZF1DX7qK8ma5wgG1",
            "37i9dQZF1DWX83CujKHHOn",
            "37i9dQZF1DWZUAeYvs88zc",
            "37i9dQZF1DX0oYNw3AgyUT",
             

            ],
            "HeartBreak"
        ),
             (
            [
            "37i9dQZF1DX9tPFwDMOaN1",
            "37i9dQZF1DX4FcAKI5Nhzq",
            "37i9dQZF1DXe5W6diBL5N4",
            "37i9dQZF1DX14fiWYoe7Oh",
            "37i9dQZF1DX4IDaXtVjL83",
            "37i9dQZF1DWW46Vfs1oltB",
            "37i9dQZF1DX2SFBzpAPi7n",
            "37i9dQZF1DX7vZYLzFGQXc",
             

            ],

            "K-Pop"
        ),
             (
            [
            "37i9dQZF1DXb3MZdETGqKB",
            "37i9dQZF1DXcnkReojaCnV",
            "37i9dQZF1DX0QNpebF7rcL",
            "37i9dQZF1DWSKpvyAAcaNZ",
            "37i9dQZF1DXaiAJKcabR16",
            "37i9dQZF1DX9stbPFTxeaB",
            "37i9dQZF1DWZBWeillXfzX",
            "37i9dQZF1DWT1RLff7HxDm",
             

            ],
           
            "Modern Blues"
        ),
             (
            [
            "37i9dQZF1DXd9rSDyQguIk",
            "37i9dQZF1DXbkKnGZHv1kf",
            "37i9dQZF1DX9GnjlK17CLK",
            "37i9dQZF1DXcu3QLJudo4X",
            "37i9dQZF1DXbxvVclJxhEJ",
            "37i9dQZF1DWWhiW4fdIska",
            "37i9dQZF1DWVKE4YNKbqtI",
            "37i9dQZF1DXas3RS9LEsSS",
             

            ],
            "Classic Blues"
        ),
             (
            [
            "37i9dQZF1DWYkaDif7Ztbp",
            "37i9dQZF1DWT6pI50XnTRh",
            "37i9dQZF1DWYs2pvwxWA7l",
            "37i9dQZF1DWSjibQnF0bUk",
            "37i9dQZF1DX0SlHbqMm3vJ",
            "37i9dQZF1DX0ItcQfwbKZJ",
            "37i9dQZF1DWVyizF9BJ61m",
             

            ],
            "Afro"
        ),
             (
            [
            "37i9dQZF1DWVYs6zNzJ0ci",
            "37i9dQZF1DX4Ux6OGNoFON",
            "37i9dQZF1DX3fXJqxGjuEP",
            "37i9dQZF1DXbSbnqxMTGx9",
            "37i9dQZF1DWSiyIBdVQrkk",
            "37i9dQZF1DXbwoaqxaoAVr",
            "37i9dQZF1DX3mvx0imueXo",
             

            ],

            "Reggae"
        ),
             (
            [
            "37i9dQZF1DWWvhKV4FBciw",
            "37i9dQZF1DXdJyxIBq7Xdy",
            "37i9dQZF1DWVqFWv4EZA70",
            "37i9dQZF1DWUS3jbm4YExP",
            "37i9dQZF1DXbjGYBfEmjR5",
            "37i9dQZF1DX6drTZKzZwSo",
            "37i9dQZF1DX4WgZiuR77Ef",
             

            ],
            "Funk & Disco"
        ),
             (
            [
            "37i9dQZF1DX1lVhptIYRda",
            "37i9dQZF1DXaiEFNvQPZrM",
            "37i9dQZF1DWZBCPUIUs2iR",
            "37i9dQZF1DWYnwbYQ5HnZU",
            "37i9dQZF1DX8S0uQvJ4gaa",
            "37i9dQZF1DX8WMG8VPSOJC",
            "37i9dQZF1DX5U9LWxuimGj",
            "37i9dQZF1DWXjt5iSjw7DI",
             

            ],
            "Popular Country"
        ),
             (
            [
            "37i9dQZF1DWXdiK4WAVRUW",
            "37i9dQZF1DWVpjAJGB70vU",
            "37i9dQZF1DXdxUH6sNtcDe",
            "37i9dQZF1DX6RCydf9ytsj",
            "37i9dQZF1DWYP5PUsVbso9",
            "37i9dQZF1DWWnpcjfCqaW0",
            "37i9dQZF1DX7CGYgLhqwu5",
            "37i9dQZF1DX0vVAWnG6fKT",
             

            ],
            "Country Decades"
        ),
    ],
    "Playlists_Spotify_v2.csv"               

)


Norteñas
Corridos
Sierreño
Grupero
Mariachi
Regeaton
Latin Pop
Tropical
Cumbia
Salsa
Classic Hip Hop
Mexican Hip Hop
Electronica & Chill
EDM & Dance Pop
'NoneType' object is not a mapping
House
Techno
Disco
Rock


HTTP Error for GET to https://api.spotify.com/v1/artists/?ids= with Params: {} returned 400 due to invalid id


http status: 400, code:-1 - https://api.spotify.com/v1/artists/?ids=:
 invalid id, reason: None
Indie


TypeError: 'NoneType' object is not subscriptable

#**Data Dictionary**#